In [10]:
# demo program for using two hidden layer with custom regularizer
# note: I don't split the train and test data, please google and modify it to fulfil your demand.

#generate XOR dataset
import numpy as np
bit = 5
X=[]
Y=[]
for i in range(2**bit):
    x='{0:05b}'.format(i)
    x=map(int,x)
    y=sum(x)%2
    X.append(x)
    Y.append(y)
X=np.array(X)
Y=np.array(Y)

import tensorflow as tf
N_x = bit
N_h1 = 10
N_h2 = 10
N_out = 2

GRAPHKEY_NAME='REGULARIZATION'

# you can try to write your own regularizer similarly
def my_regularizer(tensor,labmda=0.0001):
    return tf.mul(tf.reduce_sum(tf.abs(tensor)), labmda, name='L1-Loss')

def weight_variable(shape,std=0.1):
    """Create a weight variable with appropriate initialization."""
    initial = tf.truncated_normal(shape, stddev=std)
    return tf.Variable(initial)

def bias_variable(shape,val=0.1):
    """Create a bias variable with appropriate initialization."""
    initial = tf.constant(val, shape=shape)
    return tf.Variable(initial)

def fully_connect(name,input_tensor,indim,outdim,regularizer=None):
    with tf.name_scope(name):
        weights = weight_variable([indim, outdim])
        biases = bias_variable([outdim])
        # add custom regularization
        if regularizer:
            reg = regularizer(weights)+regularizer(biases)
            tf.add_to_collection(GRAPHKEY_NAME,reg)
        
        return tf.matmul(input_tensor, weights) + biases

def build_model(input_tensor):
    h1 = fully_connect('h1',input_tensor,N_x,N_h1,my_regularizer)
    a1 = tf.nn.relu(h1)
    
    h2 = fully_connect('h2',a1,N_h1,N_h2,my_regularizer)
    a2 = tf.nn.relu(h2)
    
    logits = fully_connect('out',a2,N_h2,N_out,my_regularizer)
    return logits

def one_hot(target,nlabel):
    with tf.variable_scope('onehot'):
        one_hot_target = tf.one_hot(tf.cast(tf.reshape(target,[-1]),tf.int32),nlabel)
    return one_hot_target

N_iter=1500
N_print = 50
with tf.Session() as sess:
    x = tf.placeholder(shape=[None,N_x],dtype = tf.float32,name='input')
    y = tf.placeholder(shape=[None],dtype=tf.float32,name='label')
    y_enc = one_hot(y,2)
    
    logit = build_model(x)
    correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(y_enc,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    diff = tf.nn.softmax_cross_entropy_with_logits(logit,y_enc)
    cross_entropy = tf.reduce_mean(diff)
    loss= cross_entropy + tf.reduce_sum(tf.get_collection(GRAPHKEY_NAME))
    
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    sess.run(tf.initialize_all_variables())
    for i in xrange(N_iter):
        _,acc = sess.run([train_step,accuracy],feed_dict={x:X,y:Y})
        if i%N_print == 0:
            print acc

0.46875
0.46875
0.5
0.5
0.53125
0.65625
0.65625
0.65625
0.65625
0.65625
0.65625
0.6875
0.6875
0.71875
0.84375
0.65625
0.90625
0.96875
0.96875
0.96875
1.0
0.96875
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
